In [1]:
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from itertools import repeat

In [2]:
with open("solutions.txt") as f:
    text = f.read()
    solution_set = [i.replace('"', '').replace(' ', '') for i in text.split(",")]
solution_set[0:5]

['cigar', 'rebut', 'sissy', 'humph', 'awake']

In [3]:
with open("nonsolutions.txt") as f:
    text = f.read()
    nonsolution_set = [i.replace('"', '').replace(' ', '') for i in text.split(",")]
nonsolution_set[0:5]

['aahed', 'aalii', 'aargh', 'aarti', 'abaca']

In [4]:
def get_response(guess, solution):
    response = ""
    for w in range(len(guess)):
        if guess[w] == solution[w]:
            response += "2"
        elif guess[w] in solution:
            response += "1"
        else:
            response += "0"
    return response

def get_possible_outcomes(guess, remaining):
    responses = [[solution, get_response(guess, solution)] for solution in remaining]
    outcomes = list(set(r[1] for r in responses))
    outcomes_with_length = [[r, len([v for v in responses if v[1] == r])] for r in outcomes]
    return outcomes_with_length

def get_score(guess, remaining):
    outcomes = get_possible_outcomes(guess, remaining)
    max_set_size = max([v[1] for v in outcomes])
    # print(outcomes)
    return [guess, max_set_size]

def get_best(remaining):
    with ThreadPoolExecutor(max_workers=15) as executor:
        word_scores = list(executor.map(get_score, remaining, repeat(remaining))) #  [[word, get_score(word, remaining)] for word in remaining]
    scores = [entry[1] for entry in word_scores]
    min_score = min(scores)
    idx = scores.index(min_score)
    return word_scores[idx][0]

def filter_words(guess, remaining, response):
    feasible_words = [i for i in remaining if get_response(guess, i) == response]
    return feasible_words

In [5]:
# get_best(solution_set)

In [6]:
filter_words('arise', solution_set, '02201')

['pried',
 'grief',
 'brief',
 'crier',
 'drier',
 'tried',
 'fried',
 'cried',
 'dried']

In [7]:
small_set = ['pried',
 'grief',
 'brief',
 'crier',
 'drier',
 'tried',
 'fried',
 'cried',
 'dried']

In [8]:
def recursive_best(remaining, ctr=1, debug=False):
    ctr = int(ctr)
    if len(remaining) == 1:
        if debug: print('  '*ctr + f"\t\t\tReturning count {ctr} for {remaining[0]}")
        return remaining[0], ctr
    
    if ctr > 6:
        return remaining[0], None

    set_scores = []
    for word in remaining:
        set_score = get_score(word, remaining)
        set_scores.append(set_score[1])
    best_set_score = min(set_scores)
    initial_filter = [set_scores[i] <= best_set_score * 1.5 for i in range(len(remaining))]

    word_scores = []
    for word_index, word in enumerate(remaining):
        if ctr == 1:
            print(f"{word_index}/{len(remaining)}")
        if debug: print('  '*ctr + f" Trying {word}")
        if initial_filter[word_index] == False:
            word_scores.append(5)
            continue
        outcomes = get_possible_outcomes(word, remaining)
        if debug: print('  '*ctr + " Outcomes:", outcomes)
        total = sum(i[1] for i in outcomes) # count case sum
        values = []
        failed = False
        for o in outcomes:
            if debug: print('  '*ctr + f"  Outcome {o[0]} with {o[1]} candidates")
            subset = filter_words(word, remaining, o[0])
            if debug: print('  '*ctr + f"  Remaining subset: {subset}")
            best_word, val = recursive_best(subset, ctr+1)
            if val is None:
                failed = True
                break
            values.append(val)
        # probability times number of avg moves
        if failed:
            print(f"Word {word} failed")
            word_scores.append(5)
            continue
        score = sum([outcomes[idx][1]/total * values[idx] for idx in range(len(outcomes))])
        word_scores.append(score)
    best_word_score = min(word_scores)
    index = word_scores.index(best_word_score)
    if debug:
        for i in range(len(remaining)):
            print(f"Word {remaining[i]} score {word_scores[i]}")
    return remaining[index], best_word_score

In [9]:
# word, avg_trial = recursive_best(solution_set[0:1000])
# print(word, avg_trial)

In [10]:
s = solution_set.copy() # Initialize

In [11]:
get_best(s)

'arise'

In [12]:
s = filter_words('arise', s, '01001')

In [13]:
get_best(s)

'outer'

In [14]:
s2 = filter_words('outer', s, '10022')

In [18]:
s2

['wooer',
 'goner',
 'foyer',
 'homer',
 'power',
 'poker',
 'lover',
 'rover',
 'cover',
 'joker',
 'lower',
 'boxer',
 'hover',
 'mover',
 'cower',
 'corer',
 'mower',
 'roger',
 'rower']

In [15]:
get_best(s2)

'cower'

In [16]:
s3 = filter_words('cower', s2, '02222')

In [17]:
s3

['power', 'lower', 'mower', 'rower']